# Sujet : Independant Set
Etant donné un corpus, l’objectif est de décrire chaque document par un ensemble de termes diversifiés ayant de fortes
valeurs TF-IDF.

Chaque document est prétraité pour détecter des termes (mots ou groupes de mots)
On sélectionne les termes ayant une valeur supérieure à un seuil fixé (paramètre du programme). On crée un graphe de similarité G entre ces termes. On
calcule ensuite plusieurs maximal independent sets de G comprenant le terme t ayant le score TF-IDF le plus élevé. 

Parmi les maximal independent sets ainsi créés, on identifie celui dont la somme des poids (les poids des sommets sont ici les valeurs TF-IDF des termes) est maximale. Un programme est ensuite créé qui permet pour un document donné
(passé comme argument) d’afficher son graphe G avec les sommets appartenant au meilleur independent set en rouge et les autres en bleu.



---



---


Ce notebook a été codé pour répondre au sujet en permettant à l'utilisateur de choisir plusieurs paramètres au fur et à mesure de l'exécution des cellules (choix du jeu de données, de la taille, du seuil de similarité ...). Cette approche nous permet de détailler chacune des étapes et nous a semblé plus claire pour expliquer notre démarche. 

## Importation des packages et téléchargements des bibliothèques

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from sklearn.datasets import fetch_20newsgroups

import nltk
from nltk import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer  

import string

from gensim import *
from gensim.utils import simple_preprocess

from pprint import pprint

from smart_open import smart_open

import os

import networkx as nx

import spacy 

#python -m spacy download en_core_web_sm
import en_core_web_sm
import math
import matplotlib.pyplot as plt
import random

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

nlp = en_core_web_sm.load()

stop_list = stopwords.words('english') 
exclude = set(string.punctuation)
lemmatizer = WordNetLemmatizer() 

## Sélection des paramètres

### Choix du jeu de données

In [ ]:
while True:
    try:
        choixdata = int(input("Vous avez le choix entre 2 jeux de données : NG20 (numero 1) et Classic3 (numero 2) \n"))
        if choixdata == 1 or choixdata == 2 : 
            break
        else :
            print("La valeur n'est pas entre 0 et 1")
            pass
    except:
        print("Ce n'est pas une entrée correcte")
        pass

### Taille du jeu de données
Attention, utiliser 100% du jeu de données peut considérablement augmenter le temps de traitement et notamment lors du calcul de la similarité

In [ ]:
while True:
    try:
        datasize = float(input("Entrez le pourcentage du jeu de données que vous voulez utiliser entre 0 et 1, le nombre doit être séparé par un point : "))
        if 0 <= datasize <= 1 : 
            break
        else :
            print("La valeur n'est pas entre 0 et 1")
            pass
    except:
        print("Ce n'est pas une entrée correcte")
        pass

### Similarité
Les arrêtes du graphe correspondent à la similarité entre deux mots. Pour un souci de lisibilité et d'interêt, il est nécessaire d'utiliser une valeur de similarité minimale.

In [ ]:
while True:
    try:
        myweights = float(input("Entrez la valeur du filtre de similarité entre 0 et 1, le nombre doit être séparé par un point : "))
        if 0 <= myweights <= 1 : 
            break
        else :
            print("La valeur n'est pas entre 0 et 1")
            pass
    except:
        print("Ce n'est pas une entrée correcte")
        pass

### Valeur du TFIDF

In [ ]:
while True:
    try:
        filTFIDF = float(input("Entrez la valeur du filtre du TFIDF entre 0 et 1, le nombre doit être separé par un point : "))
        if 0 <= filTFIDF <= 1 : 
            break
        else :
            print("La valeur n'est pas entre 0 et 1")
            pass
    except:
        print("Ce n'est pas une entrée correcte")
        pass

## Chargement de la base de données

In [ ]:
if choixdata == 1 : 
  with open("classic3_raw.txt") as file:
    content = file.read()
  content = content.split("\n")
else : 
  ng = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'))  
  content = ng.data

content = random.sample(content, int(round(len(content) * datasize)))

## Text cleaning

Pour le moment le texte est brute. Avant de pouvoir l'analyser, une première étape de nettoyage est nécessaire.   
Nous retirons donc : 
*   Les nombres
*   Les majuscules
*   Les mots de liaison
*   La ponctuation et les caractères spéciaux

La lemmatisation de chacun des mots du corpus est ensuite effectuée afin de regrouper tous les mots de même famille sous une seule et même forme, la forme canonique.   
Exemple : sleeps devient sleep






In [ ]:
reviews_corpus = []
for a in content:
    if len(a) > 1:
        a = ''.join([i for i in a if not i.isdigit()])
        a = ''.join(ch for ch in a if ch not in exclude)
        a = a.lower()
        a = ' '.join([word for word in a.split() if word not in stop_list])
        a = ' '.join([lemmatizer.lemmatize(word) for word in a.split() ])
        reviews_corpus.append(a)

Le dictionnaire des mots uniques du corpus est ensuite créé.

In [ ]:
texts = [[text for text in doc.split()] for doc in reviews_corpus]
dictionary = corpora.Dictionary(texts)

Nous comptons, ensuite, la fréquence des mots dans le corpus.

In [ ]:
tokenized_list = [simple_preprocess(doc) for doc in reviews_corpus]

mydict = corpora.Dictionary()
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_list]
word_counts = [[(mydict[id], count) for id, count in line] for line in mycorpus]

## Utilisation des mots

### TF IDF
Afin de créer le graphe, il faut calculer la valeur TF-IDF de chaque mot du corpus.

**Définition** : Cette mesure statistique permet d'évaluer l'importance d'un terme contenu dans un document, relativement à une collection ou un corpus. Le poids augmente proportionnellement au nombre d'occurrences du mot dans le document. Il varie également en fonction de la fréquence du mot dans le corpus. Des variantes de la formule originale sont souvent utilisées dans des moteurs de recherche pour apprécier la pertinence d'un document en fonction des critères de recherche de l'utilisateur.

Pour calculer le TF IDF, nous avons découpé le document en phrase. Le calcul est donc la récurrence d'un mot dans la phrase ainsi que dans le reste des phrases du document.

In [ ]:
tfidf = models.TfidfModel(mycorpus, smartirs='ntc')
tt_tfidf = []
# Show the TF-IDF weights
for doc in tfidf[mycorpus]:
    val = [[mydict[id], np.around(freq, decimals=2)] for id, freq in doc]
    tt_tfidf.append(val)

### Bigramme

Il existe plusieurs manière de créer un graphe. Dans notre cas, nous avons decidé de créer l'ensemble des combinaisons de deux mots présents dans le corpus. Pour cela, nous allons créer des bi-grammes.

In [ ]:
mylist = []
for gram in tt_tfidf : 
    for i in gram : 
        mylist.append(i)
mylist = [t for t in mylist if t[1]> filTFIDF]

In [ ]:
voc = []
for i in mylist : 
    voc.append(i[0])

In [ ]:
verif = []
test = []
for i in range(len(texts)) : 
    test.append(list(set([num for num in texts[i] if num in voc])))
test= [x for x in test if x != []]

In [ ]:
bigramme = []
for gramm in test : 
    bigram=list(ngrams(gramm,2))
    bigramme.append(bigram)

In [ ]:
fin = []
for gram in bigramme : 
    for i in gram : 
        fin.append(i)

### Indicateur de similarité
Les liens entre les mots représentent leur similarité. La force des lien dépend donc des valeurs de similarité.

A nouveau, il existe un certain nombres de méthodes allant du nombre de lettres en commun à la proportion de lettres ordonnées de la même manière. 
Nous avons utilisé la méthode de similarité implémentée dans le package Spacy.


In [ ]:
top = list()
for i in fin:
    token1 = nlp(i[0])
    token2 = nlp(i[1])
    sim = token1.similarity(token2)
    vol = (str(token1), str(token2), sim)
    top.append(vol)

Pour améliorer la lisibilité du graphe, nous mutiplions la similarité par 10, et la passons à l'exponentiel.

In [ ]:
tt = [math.exp(x[1]*11) for x in mylist]

## Création du graphe

Tout d'abord, nous regroupons les informations utiles dans une nouvelle table. Celle-ci contient une colonne pour le premier mot, une colonne pour le second et une dernière pour le poids (valeur de similarité) qui leur correspond.



In [ ]:
test = pd.DataFrame(top, columns = ["from", "to", "weights"])
tuples = [tuple(x) for x in test.to_numpy()]

### Initialisation du graph

In [ ]:
g = nx.Graph()
g.add_weighted_edges_from(tuples)
edges,weights = zip(*nx.get_edge_attributes(g,'weight').items())

### Calcul du maximum independante set

In [ ]:
maxindep = pd.DataFrame(nx.maximal_independent_set(g))
maxindep = maxindep[0].tolist()

In [ ]:
df_tfidf = pd.DataFrame(mylist)

Mise en place des couleurs

In [ ]:
color_map = []
for node in g:
    if node in maxindep:
        color_map.append('red')
    else: 
        color_map.append('blue') 

### Dessin du graphe

In [ ]:
fig = plt.figure(figsize = (30,30))
ax = plt.subplot(111)
ax.set_title('Graph')
pos = nx.spring_layout(g)
nx.draw(g, pos, node_color=color_map, 
        edgelist=edges, edge_color=weights, 
        width=10.0, edge_cmap=plt.cm.Blues, 
        node_size = tt, with_labels=True)
plt.savefig("graph.png", format="PNG")

In [ ]:
mergeindep = pd.merge(maxindep, df_tfidf, on = 0)
mergeindep[1].sum()

# Conclusion 
Ce projet nous a permi d'aborder plusieurs problématiques de text mining (nettoyage de données, volumétrie ...). 
Nous avons été confronté, par exemple, au choix entre l'utilisation de l'ensemble du jeu de données (exhaustivité et meilleure représentativité du graphe obtenu) et le temps d'exécution. Pour que l'utilisateur puisse faire ce choix lui-même nous avons proposé à plusieurs reprises de filter le jeu de données (par exemple sur les valeurs de la similarité ou celle des TF IDF). 

Pour poursuivre, il serait intéressant de calculer la similarité avec d'autres méthodes afin de comparer les performances obtenues (temps de traitement, graphe résultats).